### APAN5400 Project BookStarter : ETL Part

In [ ]:
import numpy as np
import pandas as pd
import random
import requests
import json

#### Create function BinarySearch

In [ ]:
def BinarySearch(lys, val):
    first = 0
    last = len(lys)-1
    index = -1
    while (first <= last) and (index == -1):
        mid = (first+last)//2
        if lys[mid] == val:
            index = mid
        else:
            if val<lys[mid]:
                last = mid -1
            else:
                first = mid +1
    if index == -1 : return False
    else: return True

Openlibrary dataset consists of 3 parts.

1. Work: a collection of the similar book edition. The same book like "Harry Potter and the Philosopher's Stone" might have multiple edition with different language.
2. Edition: an edition of each book which have isbn as key
3. Authors: Information about author with author key

New York Time provides APIs for list of all best seller history

API Call Limit = 4000 request per day and 10 request per minute -> Time interval 6 seconds

API Limit offset of data at 20 records per request

 https://developer.nytimes.com/docs/books-product/1/routes/lists/best-sellers/history.json/get

#### Download list of New York Time Bestseller books from API

In [ ]:
key = "ccyV1sHXrrBOAoXAJXAVAHybqbqA4QAU"
url = "https://api.nytimes.com/svc/books/v3/lists/best-sellers/history.json?api-key=" + key
r = requests.get(url) 
nyt_history_data = r.json()

all_result = nyt_history_data['num_results']

In [ ]:
numloop = all_result // 20 
list_isbn13_nyt = []
for i in range(0, numloop):
    offset = i * 20
    url = "https://api.nytimes.com/svc/books/v3/lists/best-sellers/history.json?api-key=" + key + "&offset=" + str(offset)
    r = requests.get(url) 
    history_data = r.json()
    try:
        history_data['results']
        for x in history_data['results']:
            try:
              isbn13 = x['isbns'][0]['isbn13']
            except:
              print("No isbn")
            list_isbn13_nyt.append(isbn13)
        display(offset)
        time.sleep(7) #Create time interval 7 seconds
    except:
        print(history_data)   

Write list of New York Time Bestseller books into text file

In [ ]:
with open('list_isbn13_nyt.txt', 'w') as outfile:
    for isbn13 in list_isbn13_nyt:
        outfile.write(isbn13 + ", ")

### Select random dataset from Openlibrary dataset

Due to an enormous size of record from Openlibrary dataset, we randomly select record from dataset to clean, extract and load to our database (MongoDB and ElasticSearch).

##### Select Edition which does not has ISBN13, Authors as Null and has number of page more than 50.

In [ ]:
#Check ISBN13 Null and Write new file only record that include isbn13,authors, and number of pages >= 50
filename = 'ol_dump_edition_filter_isbn13_authors_notnull.txt'
file = open(filename, "w")
mil = 1
with open('ol_dump_editions_2022-03-02.txt') as bigfile:
    for lineno, line in enumerate(bigfile):
        index_cut = line.find("{")
        line1 = line[index_cut:]
        json1 = json.loads(line1)
        try: 
            json1["isbn_13"]
            json1["authors"]
            json1["number_of_pages"]
            if(json1["number_of_pages"] >= 50):
                if lineno % 1000000 == 0: #Check Progress by print every million line
                    print(mil)
                    mil = mil + 1
                file.write(line1)
        except:
            pass
file.close()

In [ ]:
#Read list of nyt bestseller books isbn13
my_file = open("list_isbn13_nyt.txt", "r")
nyt_content = my_file.read()
nyt_content_list = nyt_content.split(",")
my_file.close()

nyt_content_list1 = [x.strip(' ') for x in nyt_content_list]
nyt_content_list1 = list(filter(None, nyt_content_list1))
nyt_content_list = sorted(nyt_content_list1)

In [ ]:
#Join OL edition dump file with NYT Bestseller books by ISBN 13 and Export txt files
filename = 'ol_dump_edition_filter_isbn13_nyt_bestseller.txt'
file = open(filename, "w")
mil = 1
with open('ol_dump_edition_filter_isbn13_authors_notnull.txt') as bigfile:
    for lineno, line in enumerate(bigfile):
        json1 = json.loads(line)
        try:
            isbn13 = json1["isbn_13"][0]
            if BinarySearch(nyt_content_list, isbn13):
                file.write(line)
        except:
            pass 
        if lineno % 1000000 == 0: #Check Progress by print every million line
            print(mil)
            mil = mil + 1
        
file.close()

In [ ]:
#Check Number of line/records of edition w/o isbn13 null
countline = 0
with open('ol_dump_edition_filter_isbn13_authors_notnull.txt') as bigfile:
    for lineno, line in enumerate(bigfile):
        countline = countline + 1

print(countline)

In [ ]:
#Create edition record with random line + Exclude isbn13 in NYT Bestseller books list
mil = 1
filename = 'ol_dump_edition_random_line_edit_100000_withoutNYTBestsellers.txt'
file = open(filename, "w")
random.seed(619)
random_edition = random.sample(range(8364970), 100000)
random_edition = sorted(random_edition)
with open('ol_dump_edition_filter_isbn13_authors_notnull.txt') as bigfile:
    for lineno, line in enumerate(bigfile):
        if BinarySearch(random_edition,lineno):
            json1 = json.loads(line)
            try:
                isbn13 = json1["isbn_13"][0]
                if BinarySearch(nyt_content_list, isbn13):
                    pass
                else:
                    file.write(line)
            except:
                pass 
        if lineno % 1000000 == 0: #Check Progress by print every million line
            print(mil)
            mil = mil + 1
file.close()

In [ ]:
#Combine Edition Random + NYT Bestseller books and Export as a file

filenames = ['ol_dump_edition_random_line_edit_100000_withoutNYTBestsellers.txt','ol_dump_edition_filter_isbn13_nyt_bestseller.txt']

with open('ol_dump_edition_random_line_with_nytbestseller.txt', 'w') as outfile:  
    # Iterate through list
    for names in filenames:
        # Open each file in read mode
        with open(names) as infile:
            # read the data from file1 and
            # file2 and write it in file3
            outfile.write(infile.read())
        # Add '\n' to enter data of file2
        # from next line
        outfile.write("\n")

##### Select Author record from OpenLibrary dumpfile randomly

In [ ]:
#Export OL Authors random
countline = 0
with open('ol_dump_authors_2022-03-02.txt') as bigfile:
    for lineno, line in enumerate(bigfile):
        countline = countline + 1

print(countline)

In [ ]:
filename = 'ol_dump_authors_random_line_edit_300000.txt'
file = open(filename, "w")
random_edition = random.sample(range(9408934), 300000)
random_edition = sorted(random_edition)
with open('ol_dump_authors_2022-03-02.txt') as bigfile:
    for lineno, line in enumerate(bigfile):
        if BinarySearch(random_edition,lineno):
            index_cut = line.find("{")
            line1 = line[index_cut:]
            file.write(line1)
            
file.close()

##### Select Work record from OpenLibrary dumpfile randomly

In [ ]:
#Export OL Works random
countline = 0
with open('ol_dump_works_2022-03-02.txt') as bigfile:
    for lineno, line in enumerate(bigfile):
        countline = countline + 1

print(countline)

In [ ]:
filename = 'ol_dump_works_random_line_edit_100000.txt'
file = open(filename, "w")
random_edition = random.sample(range(24713455), 100000)
random_edition = sorted(random_edition)
with open('ol_dump_works_2022-03-02.txt') as bigfile:
    for lineno, line in enumerate(bigfile):
        if BinarySearch(random_edition,lineno):
            index_cut = line.find("{")
            line1 = line[index_cut:]
            file.write(line1)
            
file.close()

### Load Dataset into MongoDB

In [ ]:
#!pip3 install -U pymongo
from pymongo import MongoClient
client = MongoClient('localhost',27017) ## or MongoClient("localhost:27")
db = client.apan5400project

In [ ]:
json_data = open("ol_dump_works_random_line_edit_100000.txt").readlines()
jdata1 = []
for line in json_data:
    jdata1.append(json.loads(line))
print(len(jdata1))

json_data2 = open("ol_dump_edition_random_line_with_nytbestseller.txt").readlines()
jdata2 = []
for line in json_data2:
    try:
        jdata2.append(json.loads(line))
    except:
        pass
print(len(jdata2))

json_data3 = open("ol_dump_authors_random_line_edit_300000.txt").readlines()
jdata3 = []
for line in json_data3:
    jdata3.append(json.loads(line))
print(len(jdata3))

In [ ]:
collection1 = db.works
collection1.insert_many(jdata1)

collection2 = db.editions
collection2.insert_many(jdata2)

collection3 = db.authors
collection3.insert_many(jdata3)

### Extract variables from dataset and load dataset into Elasticsearch

In [ ]:
edition_result = collection2.find ()

In [ ]:
edition_list = []

for record in edition_result: 
    edition_list.append(record)



In [ ]:
jdata4 = []
for edition in edition_list:
    line = {}
    try:
        line['title'] = edition['title']
    except:
        pass
    try:
        line['subjects'] = ", ".join(edition['subjects'])
    except:
        pass
    line['authors'] = edition['authors'][0]['key']
    line['number_of_pages'] = edition['number_of_pages']
    line['isbn_13'] = edition['isbn_13'][0]
    try:
        line['revision'] = edition['revision']
    except:
        pass
    try:
        line['latest_revision'] = edition['latest_revision']
    except:
        pass
    jdata4.append(line)

#### Extract Authors name from OpenLibrary Authors dataset

In [ ]:
all_author_name = []
mil = 1
with open('ol_dump_authors_2022-03-02.txt') as bigfile:
    for lineno, line in enumerate(bigfile):
        index_cut = line.find("{")
        line1 = line[index_cut:]
        json1 = json.loads(line1)
        try:
            json1['name']
            line_app = {}
            line_app['key'] = json1['key']
            line_app['author'] = json1['name']
            all_author_name.append(line_app)
            if lineno % 1000000 == 0: #Check Progress by print every million line
                print(mil)
                mil = mil + 1   
        except: pass

In [ ]:
list_all_author_name_key = []
for x in all_author_name:
    list_all_author_name_key.append(x['key'])

In [ ]:
list_all_author_name_author = []
for x in all_author_name:
    list_all_author_name_author.append(x['author'])

In [ ]:
set_list_all_author_name_key = set(list_all_author_name_key)

In [ ]:
jdata6 = []
mil = 1
for edition in jdata4:
    author = edition['authors']
    isbn13 = edition['isbn_13']
    line_app = {}
    try:
        line_app['title'] = edition['title']
    except: pass
    try:
        line_app['subjects'] = edition['subjects']
    except: pass
    try:
        line_app['revision'] = edition['revision']
    except: pass
    try:
        line_app['latest_revision'] = edition['latest_revision']
    except: pass
    line_app['number_of_pages'] = edition['number_of_pages']
    line_app['isbn_13'] = isbn13
    line_app['NYT_bestseller'] = BinarySearch(nyt_content_list, isbn13)
    if author in set_list_all_author_name_key:
        index = list_all_author_name_key.index(author)
        line_app['author'] = list_all_author_name_author[index]
    if mil % 1000 == 0: #Check Progress by print every million line
        print(mil)
    mil = mil + 1    
    jdata6.append(line_app)

Write list of records for importing to ElasticSearch in text file

In [ ]:
json_string = json.dumps(jdata6)
es_data = open("edition_with_authors_name_for_es_v4.json","w")
es_data.write(json_string)
es_data.close()

##### Load extract data into ElasticSearch

In [ ]:
#Change Username and Password of ElasticSearch
username = 'elastic'
password = 'b3jZ+wT*4PBtnpEN8YX4'

#Connect to Elasticsearch
from elasticsearch_dsl import connections
clientes = connections.create_connection(hosts=["localhost"],
                     port=9200, 
                     http_auth=(username, password), 
                     ca_certs='http_ca.crt',
                     use_ssl=True, 
                     verify_certs=True)

In [ ]:
#Insert Edition data to Elasticsearch
from elasticsearch.helpers import bulk

resp = bulk(clientes, jdata6, index = "edition")

### Load Open Library cover image into MongoDB

Load sample 1,000 ISBN13 from Edition collection from MongoDB Database

In [ ]:
# object = edition.find_one()
result = collection2.find ({ 'isbn_13' : { '$exists': True } })

# edition.find({ 'isbn_13' : { '$exists': True, '$ne': NULL } });

isbn_list = []

# preparing 1000 isbn for image retrieval
for i in range(1000): 
    isbn_list.append(result[i]['isbn_13'])


Create Function to download cover image from OpenLibrary APIs

In [ ]:
# function to retrieve and download the image locally
def download_image():
    for i in isbn_list:
        # prepare the isbn value and confirm size
        isbn = i[0]
        file_type = isbn + '-L.jpg'

        # putting isbn and file_tye together to form our url
        url = 'https://covers.openlibrary.org/b/isbn/' + file_type


        response = requests.get(url, stream=True)

        # save the image file using isbn as name
        save_name = isbn + '.jpg'
        with open(save_name, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        del response

Call Download Cover Function and load the data into MongoDB

In [ ]:
# upload the image to mongodb
import gridfs

#Create a object of GridFs for the above database.
image = gridfs.GridFS(db)

#define an image object with the location.
for i in isbn_list: 
    isbn = i[0]
    file = isbn + '.jpg'

    #Open the image in read-only format.
    with open(file, 'rb') as f:
        contents = f.read()

    #Now store/put the image via GridFs object.
    image.put(contents, filename=file)